# Importing package

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
filedirectory = "//Users//joancaceres//Documents//FERBO//FB_W10_S4//run2//"
import scqubits as sq
from src.utilities import *
from src.fitting import fit_fluxonium, get_transitions_from_levels, plot_transitions

# Loading data

In [2]:
voltage_period = 3.033
zero_flux = 1.3668

In [10]:
datasets = {}

In [11]:
datanumbers = [
    424403,
    422958,# Large upper scan around phase = 0
    422231, # Close look at phae = pi
    424530, # Large bottom  scan around phase = pi
]

In [12]:
for number in datanumbers:
    datasets[number] = load_data(filedirectory + "FBW10S4_" +str(number))
    datasets[number]['freq'] = datasets[number]['freq']*1e-9
    datasets[number]['phi'] = (datasets[number]['flux']-zero_flux)/voltage_period

In [14]:
fig1, ax1 = plot3Ds(
    data_dicts=datasets,
    x_key='phi',
    y_key='freq',
    z_key='phase',
    flatten_horizontal=0,
    flatten_vertical=1,
    vmin=-25,
    vmax=50,
    cmap='gray'
)

# Fluxonium visualization

In [229]:
fluxonium_params = {
    'EJ':   10.58,
    'EC':   1.79,
    'EL':   0.367,
    'flux': 0
}
fluxonium = sq.Fluxonium(**fluxonium_params,cutoff=40)

In [232]:
flux_array = np.linspace(-0.5,0.5,101)
fluxspec = fluxonium.get_spectrum_vs_paramvals(
    param_name='flux',
    param_vals=flux_array,
    evals_count=6,
    subtract_ground=False
)

Spectral data:   0%|          | 0/101 [00:00<?, ?it/s]

In [233]:
evals_matrix = fluxspec.energy_table
transitions0 = evals_matrix - evals_matrix[:,0].reshape(-1,1)
transitions1 = evals_matrix - evals_matrix[:,1].reshape(-1,1)
transitions2 = evals_matrix - evals_matrix[:,2].reshape(-1,1)

transitions0 = np.delete(transitions0,0,axis=1)
transitions1 = np.delete(transitions1,[0,1],axis=1)
transitions2 = np.delete(transitions2,[0,1,2],axis=1)

In [234]:
fig2,ax2 = plt.subplots(1,1)
ax2.plot(flux_array,transitions0, linestyle='--', color='C0', alpha=0.8)
ax2.plot(flux_array,transitions1, linestyle='--', color='C1', alpha=0.8)
plt.show()

# Fluxonium fitting

In [5]:
transition_files = [
        ('../data/data_fitting/E01.csv', 0, 1),  # transition from 0 to 1
        ('../data/data_fitting/E02.csv', 0, 2)   # transition from 0 to 2
    ]

bounds = {
        'EJ': (10, 12),  
        'EC': (1.5, 2),   
        'EL': (0.3, 0.4)      
    }

fluxonium_fit, result = fit_fluxonium(transition_files, bounds)
print("Fitted Parameters - EJ:", fluxonium_fit.EJ, "EC:", fluxonium_fit.EC, "EL:", fluxonium_fit.EL)

Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Fitted Parameters - EJ: 10.013326564128294 EC: 1.5882612500939715 EL: 0.36785158164777937


In [6]:
fluxonium_fit

Fluxonium(**{'EJ': 10.013326564128294, 'EC': 1.5882612500939715, 'EL': 0.36785158164777937, 'flux': 0, 'cutoff': 40, 'truncated_dim': 6, 'id_str': 'Fluxonium_3045', 'evals_method': None, 'evals_method_options': None, 'esys_method': None, 'esys_method_options': None})

In [353]:
fluxonium_test = sq.Fluxonium(EJ=11,EC=fluxonium_fit.EC, EL=fluxonium_fit.EL, flux=0, cutoff=40)
flux_array = np.linspace(-0.7,0.1,3001)
flux_array_test, transition_dict_test = get_transitions_from_levels(fluxonium_test, flux_array, base_levels=[0,1])
plot_transitions(flux_array, transition_dict_test, fig1, ax1, fluxonium_test)

Spectral data:   0%|          | 0/3001 [00:00<?, ?it/s]

 /Users/joancaceres/Documents/Fermionic-bosonic-qubit/notebooks/../src/fitting.py: 189

In [347]:
base_levels = [0, 1]  # Specify the base levels from which to calculate transitions
flux_array = np.linspace(-0.7,0.1,1001)
flux_array, transition_dict = get_transitions_from_levels(fluxonium_fit, flux_array, base_levels=base_levels)


Spectral data:   0%|          | 0/1001 [00:00<?, ?it/s]

In [348]:
plot_transitions(flux_array, transition_dict, fig1, ax1, fluxonium_fit)

 /Users/joancaceres/Documents/Fermionic-bosonic-qubit/notebooks/../src/fitting.py: 189

In [305]:
flux_array = np.linspace(-0.7,0.1,4001)
fluxspec = fluxonium_fit.get_spectrum_vs_paramvals(
    param_name='flux',
    param_vals=flux_array,
    evals_count=6,
    subtract_ground=False
)
evals_matrix = fluxspec.energy_table
transitions0 = evals_matrix - evals_matrix[:,0].reshape(-1,1)
transitions1 = evals_matrix - evals_matrix[:,1].reshape(-1,1)
transitions2 = evals_matrix - evals_matrix[:,2].reshape(-1,1)

transitions0 = np.delete(transitions0,0,axis=1)
transitions1 = np.delete(transitions1,[0,1],axis=1)
transitions2 = np.delete(transitions2,[0,1,2],axis=1)

Spectral data:   0%|          | 0/4001 [00:00<?, ?it/s]

In [306]:
fig1.suptitle(f'EJ = {np.round(fluxonium_fit.EJ,2)}, EC = {np.round(fluxonium_fit.EC,2)}, EL = {np.round(fluxonium_fit.EL,3)}')
alpha = 0.5
plot_tran0 = ax1.plot(flux_array,transitions0, linestyle='--', linewidth=0.8, color='C0', alpha=alpha)
plot_tran1 = ax1.plot(flux_array,transitions1, linestyle='--', linewidth=0.8, color='C1', alpha=alpha)
fig1.tight_layout()
plt.show()

In [300]:
for line in plot_tran0:
    line.remove()
for line in plot_tran1:
    line.remove()

In [359]:
from ipywidgets import interact, FloatSlider, fixed
from IPython.display import display

def update_plot(EJ, EC, EL, flux_points, base_levels, evals_count, fig1, ax1, **kwargs):
    """
    Updates the plot with new parameters for the Fluxonium object.

    Parameters:
    -----------
    EJ : float
        Josephson energy.
    EC : float
        Charging energy.
    EL : float
        Inductive energy.
    flux_points : numpy.ndarray
        Array of flux values.
    base_levels : list of int
        List of base levels from which to calculate transitions.
    evals_count : int
        Number of eigenvalues to calculate.
    fig1 : matplotlib.figure.Figure
        Existing figure to plot on.
    ax1 : matplotlib.axes._subplots.AxesSubplot
        Existing axis to plot on.
    **kwargs : dict
        Additional keyword arguments for plotting (e.g., alpha, linestyle, linewidth).
    """
    # Clear the axis to avoid overplotting
    ax1.clear()

    # Create a new Fluxonium instance with the updated parameters
    fluxonium = sq.Fluxonium(EJ=EJ, EC=EC, EL=EL, flux=0, cutoff=40)
    
    # Get the transitions from the specified base levels
    flux_array, transition_dict = get_transitions_from_levels(fluxonium, flux_points, evals_count, base_levels)
    
    # Plot the specific transitions with custom plotting parameters
    plot_transitions(flux_array, transition_dict, fig1, ax1, fluxonium, **kwargs)

### Create Interactive Widgets
def create_interactive_plot(flux_points, base_levels, fig1, ax1, linewidth, evals_count=6):
    """
    Creates an interactive plot for adjusting Fluxonium parameters.

    Parameters:
    -----------
    flux_points : numpy.ndarray
        Array of flux values.
    base_levels : list of int
        List of base levels from which to calculate transitions.
    fig1 : matplotlib.figure.Figure
        Existing figure to plot on.
    ax1 : matplotlib.axes._subplots.AxesSubplot
        Existing axis to plot on.
    evals_count : int, optional
        Number of eigenvalues to calculate (default is 6).
    """
    interact(
        update_plot,
        EJ=FloatSlider(value=11.0, min=9, max=13.0, step=0.1, description='EJ'),
        EC=FloatSlider(value=1.8, min=1, max=2.0, step=0.1, description='EC'),
        EL=FloatSlider(value=0.36, min=0.3, max=0.4, step=0.01, description='EL'),
        flux_points=fixed(flux_points),
        base_levels=fixed(base_levels),
        evals_count=fixed(evals_count),
        fig1=fixed(fig1),
        ax1=fixed(ax1),
        alpha=FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='Alpha'),
        linestyle=fixed('--'),
        linewidth=fixed(linewidth)
        )

In [361]:
# Specify the array of flux values
flux_points = np.linspace(-0.7, 0.1, 1001)
# Specify the base levels from which to calculate transitions
base_levels = [0, 1]


# Create the interactive plot
create_interactive_plot(flux_points, base_levels, fig1, ax1, linewidth=0.8)

interactive(children=(FloatSlider(value=11.0, description='EJ', max=13.0, min=9.0), FloatSlider(value=1.8, des…

In [4]:
# EJ = 5.0315 GHz
# EC = 3.1220 GHz :

# EL = 0.14348 GHz

El_to_L(0.14348e9), Ec_to_C(3.1220e9)

(1.1392634012181573e-06, 6.034339353476363e-15)